In [1]:
# Import Libraries

import numpy as np
import pandas as pd
pd.set_option('mode.chained_assignment', None)

# https://github.com/twopirllc/pandas-ta
import pandas_ta as ta    

In [2]:
# User Inputs

assetCodes = ['CL=F', 'GC=F', '^RUT', '^GSPC', 'EURUSD=X', 'GBPJPY=X', 'BTC-USD', 'ETH-USD']
assetNames = ['crudeOil', 'Gold', 'Russel2000', 'S&P500', 'EUR-USD', 'GBP-JPY', 'BTC-USD', 'ETH-USD']

# Date settings
start_date = '2006-06-30'
end_date = '2022-11-01'
#training_months = 145

# Time Interval
timePeriod="max"
daily="1d"

# Daily Indicators
dailyEMAShort = 12
dailyEMALong = 26
rsiLength = 14
momLength = 14
rocLength = 21
smaLength = 50
bbandsLength = 30
macdFast = 12
macdSlow = 26
macdSignal = 9

# Pct_Change - days ahead
days = 1

# Feature Zscore
rollingWindow = 30

# Feature Normalization
deltaDays = 1

In [3]:
# Indicator Names

# Number of Assets
count = len(assetCodes)

# Daily Indicators
dailyEMAShortIndicatorName = 'EMA_'+str(dailyEMAShort)
newDailylyEMAShortIndicatorName = 'EMAShort'

dailyEMALongIndicatorName = 'EMA_'+str(dailyEMALong)
newDailyEMALongIndicatorName = 'EMALong'

rsiIndicatorName = 'RSI_'+str(rsiLength)
newRsiIndicatorName = 'RSIline'

momIndicatorName = 'MOM_'+str(momLength)
newMomIndicatorName = 'MOMline'

rocIndicatorName = 'ROC_'+str(rocLength)
newRocIndicatorName = 'ROCline'

smaIndicatorName = 'SMA_'+str(smaLength)
newSmaIndicatorName = 'SMAline'

bollingerLowerIndicatorName = 'BBL_'+str(bbandsLength)+'_2.0'
newBollingerLowerIndicatorName = 'lowerBB'

bollingerMiddleIndicatorName = 'BBM_'+str(bbandsLength)+'_2.0'
newBollingerMiddleIndicatorName = 'middleBB'

bollingerUpperIndicatorName = 'BBU_'+str(bbandsLength)+'_2.0'
newBollingerUpperIndicatorName = 'upperBB'

bollingerStdIndicatorName = 'BBB_'+str(bbandsLength)+'_2.0'
newBollingerStdIndicatorName = '2stdBB'

macdIndicatorName = "MACD_"+str(macdFast)+"_"+str(macdSlow)+"_"+str(macdSignal)
newMACDIndicatorName = "MACDline"

macdHistogramIndicatorName = "MACDh_"+str(macdFast)+"_"+str(macdSlow)+"_"+str(macdSignal)
newMACDHistogramIndicatorName = "MACDHistogram"

macdSignalIndicatorName = "MACDs_"+str(macdFast)+"_"+str(macdSlow)+"_"+str(macdSignal)
newMACDSignalIndicatorName = "MACDSignal"

featuresRaw = [newDailylyEMAShortIndicatorName, newDailyEMALongIndicatorName, newRsiIndicatorName, newMomIndicatorName, newRocIndicatorName, 
               newSmaIndicatorName, newBollingerLowerIndicatorName, newBollingerMiddleIndicatorName, newBollingerUpperIndicatorName, 
               newBollingerStdIndicatorName, newMACDIndicatorName, newMACDHistogramIndicatorName, newMACDSignalIndicatorName]


# Number of RawFeatures
featuresCount = len(featuresRaw)

In [4]:
# Function to calculate daily Indicators

def dailyIndicators(assetCode, assetName, timePeriod, daily, days, start_date, end_date, dailyEMAShort, dailyEMALong, rsiLength, momLength, rocLength, 
                    smaLength, bbandsLength, macdFast, macdSlow, macdSignal, dailyEMAShortIndicatorName, dailyEMALongIndicatorName,
                    rsiIndicatorName, momIndicatorName, rocIndicatorName, smaIndicatorName, bollingerLowerIndicatorName, bollingerMiddleIndicatorName, bollingerUpperIndicatorName, bollingerStdIndicatorName, macdIndicatorName,
                    macdHistogramIndicatorName, macdSignalIndicatorName, newDailylyEMAShortIndicatorName,
                    newDailyEMALongIndicatorName, newRsiIndicatorName, newMomIndicatorName, newRocIndicatorName,newSmaIndicatorName,
                    newBollingerLowerIndicatorName, newBollingerMiddleIndicatorName, newBollingerUpperIndicatorName,
                    newBollingerStdIndicatorName, newMACDIndicatorName, newMACDHistogramIndicatorName,
                    newMACDSignalIndicatorName):
    
    # Get Daily Asset Data

    dfDaily = pd.DataFrame()
    dfDaily = dfDaily.ta.ticker(assetCode, period=timePeriod, interval=daily)
    dfDaily = dfDaily[(dfDaily.index > start_date) & (dfDaily.index < end_date)]
    dfDaily = dfDaily[(dfDaily.Close > 0)]
    
    # Create Ticker Column
    #dfDaily["Ticker_index"] = assetName
    dfDaily["Ticker"] = assetName
    #dfDaily["Date_index"] = dfDaily.index
    dfDaily["Date"] = dfDaily.index

    # Use the pct_change function to generate returns from close prices
    dfDaily["ActualReturns"] = dfDaily["Close"].pct_change(days).shift(-days)
    
    # Drop all NaN values from the DataFrame
    dfDaily = dfDaily.dropna()

    # Initialize the new Signal column
    dfDaily['Signal'] = 0.0

    # When Actual Returns are greater than or equal to 0, generate signal to buy asset long
    dfDaily.loc[(dfDaily['ActualReturns'] >= 0), 'Signal'] = 1

    # When Actual Returns are less than 0, generate signal to sell asset short
    dfDaily.loc[(dfDaily['ActualReturns'] < 0), 'Signal'] = -1


    # Create your own Custom Strategy
    CustomStrategyDaily = ta.Strategy(
        name="Daily Indicators",
        description="daily Trading Indicators",
        ta=[
            {"kind": "ema", "length": dailyEMAShort},
            {"kind": "ema", "length": dailyEMALong},
            {"kind": "rsi", "length": rsiLength},
            {"kind": "mom", "length": momLength},
            {"kind": "roc", "length": rocLength},
            {"kind": "sma", "length": smaLength},
            {"kind": "bbands", "length": bbandsLength},
            {"kind": "macd", "fast": macdFast, "slow": macdSlow, "signal": macdSignal},
        ]
    )


    # Run "Custom Daily Strategy"
    dfDaily.ta.strategy(CustomStrategyDaily)
    dfDaily=dfDaily.dropna()
    algoDataDaily = dfDaily[['Ticker', 'Date', 'Close', 'ActualReturns','Signal',dailyEMAShortIndicatorName, dailyEMALongIndicatorName,
                         rsiIndicatorName, momIndicatorName, rocIndicatorName, smaIndicatorName, bollingerLowerIndicatorName,
                         bollingerMiddleIndicatorName, bollingerUpperIndicatorName, bollingerStdIndicatorName,
                         macdIndicatorName, macdHistogramIndicatorName, macdSignalIndicatorName]]

    algoDataDaily = algoDataDaily.rename({dailyEMAShortIndicatorName: newDailylyEMAShortIndicatorName,
                            dailyEMALongIndicatorName: newDailyEMALongIndicatorName,
                            rsiIndicatorName: newRsiIndicatorName,
                            momIndicatorName: newMomIndicatorName,
                            rocIndicatorName: newRocIndicatorName,
                            smaIndicatorName: newSmaIndicatorName,
                            bollingerLowerIndicatorName: newBollingerLowerIndicatorName,
                            bollingerMiddleIndicatorName: newBollingerMiddleIndicatorName,
                            bollingerUpperIndicatorName: newBollingerUpperIndicatorName,
                            bollingerStdIndicatorName: newBollingerStdIndicatorName,
                            macdIndicatorName: newMACDIndicatorName,
                            macdHistogramIndicatorName: newMACDHistogramIndicatorName,
                            macdSignalIndicatorName: newMACDSignalIndicatorName}, axis=1)
    
    return algoDataDaily

In [5]:
# Calculate Feature Z_Scores
def Zscore(algoData, featuresRaw, rollingWindow):

    for i in featuresRaw:
        algoData[i+'_zscore'] = (algoData[i] - algoData[i].rolling(window=rollingWindow).mean())/algoData[i].rolling(window=rollingWindow).std()
        algoData = algoData.ffill(axis = 0)
        
    return algoData


# Normalize Features with Price
def Normalize(algoData, featuresRaw, deltaDays):

    for i in featuresRaw:
        
        if((i=='EMAShort') | (i=='EMALong') | (i=='SMAline')):
        
            algoData[i+'_normal'] = (algoData[i].diff(periods=deltaDays))
            
        elif((i=='lowerBB') | (i=='middleBB') | (i=='upperBB')):
            
            algoData[i+'_normal'] = (algoData['Close']/algoData[i])
            
        elif ((i=='RSIline') | (i=='MOMline') | (i=='ROCline') ):
            
            algoData[i+'_normal'] = (algoData[i]/algoData['2stdBB'])
            
        elif ((i=='MACDline') | (i=='MACDSignal') | (i=='MACDHistogram')):
            
            algoData[i+'_normal'] = (algoData[i])
        
        elif((i == '2stdBB')):
            
            algoData[i+'_normal'] = (algoData[i]/algoData['Close'])
        
        else:
            algoData[i+'_normal'] = 0
        
        
    algoData = algoData.ffill(axis = 0)
    
    return algoData

  
# Save file
def saveFile(algoData, assetName, days):
    fileName = 'algoData_'+assetName+'.csv'
    algoData.to_csv(fileName)

In [6]:
# Get Technical Analysis and Signal Data for all Assets

for i in range(0,count):

    algoData = dailyIndicators(assetCodes[i], assetNames[i], timePeriod, daily, days, start_date, end_date, dailyEMAShort, dailyEMALong, rsiLength, momLength, rocLength, 
                    smaLength, bbandsLength, macdFast, macdSlow, macdSignal, dailyEMAShortIndicatorName, dailyEMALongIndicatorName,
                    rsiIndicatorName, momIndicatorName, rocIndicatorName, smaIndicatorName, bollingerLowerIndicatorName, bollingerMiddleIndicatorName, bollingerUpperIndicatorName, bollingerStdIndicatorName, macdIndicatorName,
                    macdHistogramIndicatorName, macdSignalIndicatorName, newDailylyEMAShortIndicatorName,
                    newDailyEMALongIndicatorName, newRsiIndicatorName, newMomIndicatorName, newRocIndicatorName,newSmaIndicatorName,
                    newBollingerLowerIndicatorName, newBollingerMiddleIndicatorName, newBollingerUpperIndicatorName,
                    newBollingerStdIndicatorName, newMACDIndicatorName, newMACDHistogramIndicatorName,
                    newMACDSignalIndicatorName)


    
    # Convert features into rolling z-scores
    algoData = Zscore(algoData, featuresRaw, rollingWindow)
    
    # Normalize features by taking features delta divided by closing price
    algoData = Normalize(algoData, featuresRaw, deltaDays)
    
    # Drop NaN's
    algoData = algoData.dropna()
    

    # Save algoData file
    saveFile(algoData, assetNames[i], days)
    